In [96]:
import numpy as np
from scipy.ndimage.measurements import label
from threading import Thread
from queue import Queue
from skimage.feature import match_template
from willitfit.plotting.plotter import plot_all
from willitfit.optimizers.volumeoptimizer import binarize_space
from skimage.util.shape import view_as_blocks

In [97]:
VOL_UNAVAILABLE = -1
VOL_BORDER = 1
VOL_INTERIOR = -2
VOL_EMPTY = 0 

The following are just dummy data sets to run the algorithm

In [98]:
volume_space = np.full((4,4,5), VOL_EMPTY, dtype=int)
volume_space[0,:,:] = VOL_UNAVAILABLE
volume_space[:,0:2,:] = VOL_UNAVAILABLE
volume_space[0:1,:,0:1] = VOL_UNAVAILABLE
volume_space[0:1,2,1] = VOL_EMPTY
volume_space[2,2,0:2] = VOL_BORDER
volume_space[1,1,2] = VOL_BORDER

In [99]:
volume_space[1,1,0] = 5
volume_space[2,1,0] = 5
volume_space[1,2,0] = 5
volume_space[0,1,0] = 5
volume_space[1,1,1] = 5
volume_space[1,2,1] = 5
volume_space[2,1,1] = 5

In [100]:
volume_space

array([[[-1, -1, -1, -1, -1],
        [ 5, -1, -1, -1, -1],
        [-1,  0, -1, -1, -1],
        [-1, -1, -1, -1, -1]],

       [[-1, -1, -1, -1, -1],
        [ 5,  5,  1, -1, -1],
        [ 5,  5,  0,  0,  0],
        [ 0,  0,  0,  0,  0]],

       [[-1, -1, -1, -1, -1],
        [ 5,  5, -1, -1, -1],
        [ 1,  1,  0,  0,  0],
        [ 0,  0,  0,  0,  0]],

       [[-1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0]]])

In [104]:
((volume_space[2-1:-1,:,:] == 5) & (volume_space[:-2,:,:] == 5) & (volume_space[2:,:,:] == 5))

array([[[False, False, False, False, False],
        [ True, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False]],

       [[False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False]]])

In [363]:
((volume_space[:,:-1,:] == 5) & (volume_space[:,1:,:] == 5))

array([[[False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [ True,  True, False, False]],

       [[False, False, False, False],
        [False, False, False, False]]])

In [364]:
((volume_space[:,:,:-1] == 5) & (volume_space[:,:,1:] == 5))

array([[[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [ True, False, False],
        [ True, False, False]],

       [[False, False, False],
        [ True, False, False],
        [False, False, False]]])

In [240]:
np.rot90(np.where((volume_space[:-1,:,:] == 5) & (volume_space[1:,:,:] == 5)),k=1)

array([[1, 1, 1],
       [1, 1, 0],
       [0, 1, 0]])

In [356]:
np.stack(np.where((volume_space[:-1,:,:] == 5) & (volume_space[1:,:,:] == 5)))

array([[0, 1, 1],
       [1, 1, 1],
       [0, 0, 1]])

In [305]:
x = np.rot90(np.stack(np.where((volume_space[:-1,:,:] == 5) & (volume_space[1:,:,:] == 5))),k=1, axes=(1,0))

In [306]:
y = np.rot90(np.stack(np.where((volume_space[:,:-1,:] == 5) & (volume_space[:,1:,:] == 5))),k=1, axes=(1,0))

In [30]:
z = np.rot90(np.stack(np.where((volume_space[:,:,:-1] == 5) & (volume_space[:,:,1:] == 5))),k=1, axes=(1,0))

In [308]:
x

array([[0, 1, 0],
       [0, 1, 1],
       [1, 1, 1]])

In [309]:
y

array([[0, 1, 1],
       [1, 1, 1]])

In [31]:
z

array([[0, 1, 1],
       [0, 2, 1],
       [0, 1, 2]])

In [87]:
volume_space[...]

array([[[-1, -1, -1, -1],
        [ 5, -1, -1, -1],
        [-1,  0, -1, -1]],

       [[-1, -1, -1, -1],
        [ 5,  5,  1, -1],
        [ 5,  5,  0,  0]],

       [[-1, -1, -1, -1],
        [ 5,  5, -1, -1],
        [ 1,  1,  0,  0]]])

In [71]:
for (x,y,z), val in np.ndenumerate(volume_space):
    print(volume_space[x:x+2, y:y+2, z:z+2].shape)
    print("a")

(2, 2, 2)
a
(2, 2, 2)
a
(2, 2, 1)
a
(2, 2, 2)
a
(2, 2, 2)
a
(2, 2, 1)
a
(2, 1, 2)
a
(2, 1, 2)
a
(2, 1, 1)
a
(2, 2, 2)
a
(2, 2, 2)
a
(2, 2, 1)
a
(2, 2, 2)
a
(2, 2, 2)
a
(2, 2, 1)
a
(2, 1, 2)
a
(2, 1, 2)
a
(2, 1, 1)
a
(1, 2, 2)
a
(1, 2, 2)
a
(1, 2, 1)
a
(1, 2, 2)
a
(1, 2, 2)
a
(1, 2, 1)
a
(1, 1, 2)
a
(1, 1, 2)
a
(1, 1, 1)
a


In [8]:
binarize_space(volume_space)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 1, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [1, 1, 1]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 1]]])

In [18]:
pattern = np.zeros((2,2,2), dtype=int)
pattern

array([[[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]]])

In [47]:
a = np.array([1,1,3,1,5,6], dtype=int)
p = np.array([2,4], dtype=int)

In [48]:
a

array([1, 1, 3, 1, 5, 6])

In [49]:
x = p.shape[0]
x

2

In [51]:
(a == 1)

array([ True,  True, False,  True, False, False])

In [16]:
np.where(binarize_space(volume_space) == pattern)

<ipython-input-16-35883e6fb023>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.where(binarize_space(volume_space) == pattern)


(array([], dtype=int64),)

In [9]:
print(np.where((binarize_space(volume_space)[:,:,:-1] == EDGE_STRUCTURE[0]) & (binarize_space(volume_space)[:,:,1:] == EDGE_STRUCTURE[1])))
print(np.where((binarize_space(volume_space)[:,:,:-1] == EDGE_STRUCTURE[1]) & (binarize_space(volume_space)[:,:,1:] == EDGE_STRUCTURE[0])))
print(np.where((binarize_space(volume_space)[:,:-1:,] == EDGE_STRUCTURE[0]) & (binarize_space(volume_space)[:,1:,:] == EDGE_STRUCTURE[1])))
print(np.where((binarize_space(volume_space)[:,:-1:,] == EDGE_STRUCTURE[1]) & (binarize_space(volume_space)[:,1:,:] == EDGE_STRUCTURE[0])))
print(np.where((binarize_space(volume_space)[:-1,:,:] == EDGE_STRUCTURE[0]) & (binarize_space(volume_space)[1:,:,:] == EDGE_STRUCTURE[1])))
print(np.where((binarize_space(volume_space)[:-1,:,:] == EDGE_STRUCTURE[1]) & (binarize_space(volume_space)[1:,:,:] == EDGE_STRUCTURE[0])))
#print(np.where((np.rot90(binarize_space(volume_space), k=2, axes=(1,2))[:-1] == EDGE_STRUCTURE[0]) & (np.rot90(binarize_space(volume_space), k=2, axes=(1,2))[1:] == EDGE_STRUCTURE[1])))
#print(np.where((np.rot90(binarize_space(volume_space), k=2, axes=(1,2))[:-1] == EDGE_STRUCTURE[1]) & (np.rot90(binarize_space(volume_space), k=2, axes=(1,2))[1:] == EDGE_STRUCTURE[0])))
#print(np.where((np.rot90(binarize_space(volume_space), k=2, axes=(0,2))[:-1] == EDGE_STRUCTURE[0]) & (np.rot90(binarize_space(volume_space), k=2, axes=(0,2))[1:] == EDGE_STRUCTURE[1])))
#print(np.where((np.rot90(binarize_space(volume_space), k=2, axes=(0,2))[:-1] == EDGE_STRUCTURE[1]) & (np.rot90(binarize_space(volume_space), k=2, axes=(0,2))[1:] == EDGE_STRUCTURE[0])))
#print(np.where((np.rot90(binarize_space(volume_space), k=2, axes=(0,1))[:-1] == EDGE_STRUCTURE[0]) & (np.rot90(binarize_space(volume_space), k=2, axes=(0,1))[1:] == EDGE_STRUCTURE[1])))
#print(np.where((np.rot90(binarize_space(volume_space), k=2, axes=(0,1))[:-1] == EDGE_STRUCTURE[1]) & (np.rot90(binarize_space(volume_space), k=2, axes=(0,1))[1:] == EDGE_STRUCTURE[0])))


NameError: name 'EDGE_STRUCTURE' is not defined

In [ ]:
axes_list = [(1,2),(0,2)]
surfaces = 0
# Original orientation
surfaces += len(np.where((binarize_space(volume_space)[:-1] == EDGE_STRUCTURE[0]) & (binarize_space(volume_space)[1:] == EDGE_STRUCTURE[1]))[0])
# Opposite direction
surfaces += len(np.where((binarize_space(volume_space)[:-1] == EDGE_STRUCTURE[1]) & (binarize_space(volume_space)[1:] == EDGE_STRUCTURE[0]))[0])
# Now the same but rotated into two other plains
for axes in axes_list:
    # Original orientation
    surfaces += len(np.where((np.rot90(binarize_space(volume_space), k=2, axes=axes)[:-1] == EDGE_STRUCTURE[0]) & (np.rot90(binarize_space(volume_space), k=1, axes=axes)[1:] == EDGE_STRUCTURE[1]))[0])
    # Opposite direction
    surfaces += len(np.where((np.rot90(binarize_space(volume_space), k=2, axes=axes)[:-1] == EDGE_STRUCTURE[1]) & (np.rot90(binarize_space(volume_space), k=1, axes=axes)[1:] == EDGE_STRUCTURE[0]))[0])
surfaces

In [ ]:
volume_space = np.rot90(volume_space, k=1, axes=(0,1))

In [ ]:
print(np.where((binarize_space(volume_space)[:-1] == EDGE_STRUCTURE[0]) & (binarize_space(volume_space)[1:] == EDGE_STRUCTURE[1])))
#print(np.where((binarize_space(volume_space)[:-1] == EDGE_STRUCTURE[1]) & (binarize_space(volume_space)[1:] == EDGE_STRUCTURE[0])))
for rotation in VOLUME_ROTATIONS:
    print(np.where((np.rot90(binarize_space(volume_space), k=rotation[0], axes=rotation[1])[:-1] == EDGE_STRUCTURE[0]) & (np.rot90(binarize_space(volume_space), k=rotation[0], axes=rotation[1])[1:] == EDGE_STRUCTURE[1])))
flip_space = np.flip(volume_space)
print(np.where((binarize_space(flip_space)[:-1] == EDGE_STRUCTURE[0]) & (binarize_space(flip_space)[1:] == EDGE_STRUCTURE[1])))
for rotation in VOLUME_ROTATIONS:
    print(np.where((np.rot90(binarize_space(flip_space), k=rotation[0], axes=rotation[1])[:-1] == EDGE_STRUCTURE[0]) & (np.rot90(binarize_space(flip_space), k=rotation[0], axes=rotation[1])[1:] == EDGE_STRUCTURE[1])))

In [ ]:
volume_space = np.full((3,3,3), VOL_UNAVAILABLE, dtype=int)
volume_space[0,0,0] = VOL_EMPTY

In [ ]:
structure = np.array([-1,0])
structure

In [ ]:
volume_space

In [ ]:
rot_space = np.rot90(volume_space,k=3,axes=(0,2))
np.where((rot_space[:-1] == structure[0]) & (rot_space[1:] == structure[1]))

## General fit

In [ ]:
def binarize_space(volume_space):
    '''
    Sets empty space to 1 and rest to 0.
    Needed to be able to identify clusters.
    '''
    arbitrary_constant = max(VOL_UNAVAILABLE, VOL_BORDER, VOL_INTERIOR, VOL_EMPTY) + 1
    first_step = np.where(volume_space != VOL_EMPTY, arbitrary_constant, volume_space)
    second_step = np.where(first_step == VOL_EMPTY, 1, first_step)
    return np.where(second_step == arbitrary_constant, 0, second_step)

### Overall volume fit

In [ ]:
def calculate_package_volume(package):
    '''
    Returns volume in cm cubed for package
    '''
    return np.prod(package)

In [ ]:
def find_total_package_volume(article_list):
    '''
    Establishes total volume of packages in list in cm cubed.
    Loops through each article in article_list, and each package in article.
    Multiplies length, height and width (elements 1-3).
    Sums up individual products and returns them as int
    '''
    return sum([calculate_package_volume(package[1:4]) for article in article_list for package in article[2]])

In [ ]:
find_total_package_volume(article_list)

In [ ]:
def find_available_space(volume_space):
    '''
    Returns total available volume in cm cubed by subtracting non-zero values from total array size
    '''
    return volume_space.size-np.count_nonzero(binarize_space(volume_space-VOL_UNAVAILABLE))

In [ ]:
#find_available_space(volume_space)

In [ ]:
def is_space_sufficient(article_list, volume_space):
    '''
    Returns True if package volume is smaller than or equal to available volume
    '''
    return find_total_package_volume(article_list) <= find_available_space(volume_space)

In [ ]:
is_space_sufficient(article_list, volume_space)

### Fit by longest dimension

In [ ]:
def find_longest_package_dimension(article_list):
    '''
    Finds single longest dimension in cm among all packages.
    '''
    return max([max(package[1:4]) for article in article_list for package in article[2]])

In [ ]:
find_longest_package_dimension(article_list)

In [ ]:
# Linear structures in all three axes
structure_y = [
    [[0, 0, 0],
     [0, 0, 0],
     [0, 0, 0]],
    [[0, 1, 0],
     [0, 1, 0],
     [0, 1, 0]],
    [[0, 0, 0],
     [0, 0, 0],
     [0, 0, 0]]]
structure_x = np.rot90(structure_y, axes=(1,2))
structure_z = np.rot90(structure_y, axes=(0,1))
structures = [structure_x, structure_y, structure_z]

In [ ]:
def find_longest_space_dimension(volume_space):
    '''
    Determines the longest continuous stretch of empty space in all three dimensions.
    '''
    # Run along each dimension
    results = []
    for structure in structures:
        # Binarize the space, then label along given dimension
        dim_result = label(input = binarize_space(volume_space), structure=structure)[0]
        # Turn into a 1-dimensional array
        dim_result = np.reshape(dim_result, dim_result.size)
        # Remove zeros
        dim_result = np.delete(dim_result, np.where(dim_result == 0))
        # Find most frequent occurrence
        most_freq = np.bincount(dim_result).argmax()
        # Return number of occurrences, which is equal to dimension in cm
        results.append(np.count_nonzero(dim_result == most_freq))
    # Return largest value from the three dimensions
    return max(results)

In [ ]:
def is_longest_dimension_sufficient(article_list, volume_space):
    '''
    Returns True if longest package dimension is smaller than or equal to longest space dimension
    '''
    return find_longest_package_dimension(article_list) <= find_longest_space_dimension(volume_space)

## Penalizer/Scorer

In [ ]:
def find_empty_space(volume_space):
    '''
    Identifies contiguous pockets of empty space.
    Returns number of empty spaces, their total volume and the volume of the largest empty space.
    '''
    # Set up structure - this treats as one contiguous space when...
    # ...there are diagonal gaps as it might make the stack unstable
    structure = np.ones((3,3,3), dtype=int)
    # Binarize the space, then label pockets
    dim_result = label(input = binarize_space(volume_space), structure=structure)[0]
    # How many empty spaces are there?
    number_spaces = np.max(dim_result)
    # What's the total volume of empty space?
    empty_volume = np.count_nonzero(dim_result)
    # What's the volume of the largest empty space?
    largest_volume = np.count_nonzero(dim_result == number_spaces)
    return number_spaces, empty_volume, largest_volume

In [ ]:
find_empty_space(volume_space)

In [ ]:
def score_space(volume_space, article_list):
    '''
    This function assigns a score to the empty space left by the optimization algorithm.
    There is a penalty for having more empty spaces as well as a relatively smaller largest empty space.
    '''
    # Get number of empty spaces, their total volume and the volume of the largest empty space
    number_spaces, empty_volume, largest_volume = find_empty_space(volume_space)
    # Get maximum possible empty space (assumes everything can be stacked perfectly)
    max_largest_volume = find_available_space(volume_space) - find_total_package_volume(article_list)
    '''INITIAL IDEA:
    - Linear penalty for each empty space
    - Log penalty for how much largest empty space is smaller than maximum possible
    '''
    score = number_spaces + np.log(max(max_largest_volume - largest_volume,0.001))
    return score

## Optimizer

### Sort packages

In [ ]:
def sort_packages(article_list, sort_by="volume", direction="descending"):
    '''
    Returns 2-dimensional list of packages sorted by sort_by in the shape of
    [(article_code, article_id, package_id)]
    '''
    # Initialize list
    return_list = []
    # Loop through articles
    for article in article_list:
        # If articles exist more than once, loop here too
        for article_id in range(article[1]):
            # Loop through packages
            for package in article[2]:
                # So far only sorting by volume has been implemented.
                if sort_by == "volume":      
                    return_list.append((article[0], article_id+1, package[0], calculate_package_volume(package)))
    # Turn into a numpy array for easier sorting
    return_list = np.array(return_list)
    # Sort by volume (4th column)
    return_list = return_list[return_list[:,3].astype(int).argsort()]
    # This gives an ascending order. Reverse if needed
    if direction == "descending":
        return_list = np.flip(return_list, axis=0)
    # Remove volume column
    return return_list[:,:3]

In [ ]:
sort_packages(article_list)

We could easily have other sorting functions, or some randomness:

In [ ]:
def hash_list(article_list):
    '''
    Creates an element-wise list of hashes
    '''
    return [hash(article.tobytes()) for article in article_list]

In [ ]:
def generate_package_lists(article_list, sorters = ["volume|ascending", "volume|ascending", "volume|descending"], random_lists = 10):
    '''
    Returns list of package lists for the optimizer.
    Lists can be pre-defined or randomized.
    '''
    
    '''
    IMPROVEMENT OPTION:
    - if random_lists is larger than maximum number of permutations, just use permutations directly
    '''
    
    
    package_lists = []
    # First generate pre-defined lists
    sorters = set(sorters)
    for sorter in sorters:
        # Split by delimiter
        sort_by, direction = sorter.split("|")
        package_lists.append(sort_packages(article_list, sort_by=sort_by, direction=direction))
    
    # Add random lists
    # But first check if any are needed
    if random_lists == 0:
        return package_lists
    # Create starting point if there is none, i.e. no pre-defined lists
    if len(package_lists) == 0:
        starter_list = sort_packages(article_list)
    else:
        starter_list = np.copy(package_lists[0])
    
    # What is the actual maximum number of unique lists?
    max_permut = np.math.factorial(len(starter_list))
    # Now add as many random lists as needed    
    while True:
        # Shuffle starter list and copy data
        np.random.shuffle(starter_list)
        new_list = np.copy(starter_list)
        # Check if this particular permutation exists already by looking at hashes
        if hash_list([new_list])[0] not in hash_list(package_lists):
            package_lists.append(new_list)
        # Check if length requirement (smaller of pre-defined lists + random_lists and max_permut) is fulfilled
        if len(package_lists) == min(len(sorters) + random_lists, max_permut-2):
            break
    return package_lists

In [ ]:
generate_package_lists(article_list, random_lists=5)

### Package orientation

In [ ]:
def choose_orientation(package_length, package_width, package_height):
    '''
    A 3-dimensional object (with exceptions like cubes) that needs to align...
    ...with the grid can have six degrees of freedom.
    Returns one random orientation for a given package.
    '''
    dimensions = [package_length, package_width, package_height]
    np.random.shuffle(dimensions)
    return (dimensions[0], dimensions[1], dimensions[2])

### Optimizer function

In [ ]:
INSUFFICIENT_SPACE = "Not enough space for packages in chosen trunk."
INSUFFICIENT_DIMENSION = "At least one package dimension exceeds trunk dimension."
OPT_INSUFFICIENT_SPACE = "Optimizer could not place this package (internal return code)."
OPT_UNSUCCESSFUL = "Optimizer was unable to place all packages."

In [ ]:
def find_first_space(package_x, package_y, package_z, volume_space):
    '''
    Finds first suitable space for package dimensions.
    Returns x,y,z starting coordinates.
    '''
    # Binarize volume space - empty areas shown as ones
    bin_space = binarize_space(volume_space)
    # Initialize template shape, to be used to find in bin_space
    template_shape = np.ones((package_x, package_y, package_z))
    # No idea why that is required (maybe otherwise similarity is too high?)
    template_shape[1,1,1] = 1.01
    # Use skimage's match_template to process a sliding window over bin_space...
    # ...and find the first location of zero, which indicates the starting point
    #return match_template(bin_space, template_shape)
    result = np.where(match_template(bin_space, template_shape) == 0)
    # From here we can extract the coordinates
    for i, _ in enumerate(result[0]):
        if bin_space[result[0][i], result[1][i], result[2][i]] == 1:
            return (result[0][i], result[1][i], result[2][i])
    # If the package cannot be placed, return error code
    return OPT_INSUFFICIENT_SPACE

In [ ]:
def place_package(package_dimensions, volume_space):
    '''
    Attempt to place next package.
    If successful, return filled volume_space and start/end coordinates,
    if not, return error code.
    '''
    # Unpack package dimensions
    package_x, package_y, package_z = package_dimensions
    package_volume = calculate_package_volume(package_dimensions)
    # Find space for package
    return_value = find_first_space(package_x, package_y, package_z, volume_space)
    # Check for error codes
    if return_value == OPT_INSUFFICIENT_SPACE:
        return OPT_INSUFFICIENT_SPACE
    x, y, z  = return_value
    # Check available space - there need to be as many zeros as the package_volume for it to fit
    available_space = volume_space[x:x+package_x, y:y+package_y, z:z+package_z]
    # If the package cannot be placed, return
    if package_volume != available_space.size - np.count_nonzero(available_space):
        return OPT_INSUFFICIENT_SPACE
    # Otherwise populate the array
    # Surfaces first
    # z-plain
    volume_space[x:x+package_x, y:y+package_y, z] = VOL_BORDER
    volume_space[x:x+package_x, y:y+package_y, z+package_z-1] = VOL_BORDER
    # y-plain
    volume_space[x:x+package_x, y, z:z+package_z] = VOL_BORDER
    volume_space[x:x+package_x, y+package_y-1, z:z+package_z] = VOL_BORDER
    # x-plain
    volume_space[x, y:y+package_y, z:z+package_z] = VOL_BORDER
    volume_space[x+package_x-1, y:y+package_y, z:z+package_z] = VOL_BORDER
    # Fill interior area
    volume_space[x+1:x+package_x-1, y+1:y+package_y-1, z+1:z+package_z-1] = VOL_INTERIOR
    
    return volume_space, x, y, z, x+package_x-1, y+package_y-1, z+package_z-1

In [ ]:
def optimizer(package_list, article_list, volume_space, queue=None, max_attempts=10):
    '''
    Places packages sequentially into available space.
    If at any point a package can no longer be placed, try again up to max_attempts times.
    Returns score, attempts taken, filled volume_space and package coordinates.
    '''
    # Attempts taken
    attempts_counter = 1
    # Take pristine copy of volume_space
    empty_space = np.copy(volume_space)
    # Package counter variable
    package_counter = 0
    # Package coordinates
    package_coordinates = []
    
    # Loop while there are still packages to place
    while True:
        # Pick up first package
        package = package_list[package_counter]
        # Find article index
        idx = sum([i for i, article in enumerate(article_list) if article[0] == package[0]])
        # Get package dimensions: Looks up article by its index (idx),
        # package definitions are the 3rd element (2),
        # packages are listed, so it's the n-1th element.
        pkg = article_list[idx][2][int(package[2])-1]
        package_length, package_width, package_height = pkg[1], pkg[2], pkg[3]
        # Obtain package orientation (random)
        package_dimensions = choose_orientation(package_length, package_width, package_height)
        
        # Attempt to place package in space
        placement_result = place_package(package_dimensions, volume_space)
        
        # Check if placement was successful
        if placement_result != OPT_INSUFFICIENT_SPACE:
            # Extract return variables and append to package_coordinates
            volume_space, x_start, y_start, z_start, x_end, y_end, z_end = placement_result
            return_list = [package[0], package[1], package[2], x_start, y_start, z_start, x_end, y_end, z_end]
            package_coordinates.append(return_list)
        else:
            # If unsuccessful, increase counter of attempts taken
            attempts_counter +=1
            # See if this is too many attempts already
            if attempts_counter > max_attempts:
                # Return error code
                return OPT_UNSUCCESSFUL
            else:
                # Try again
                package_counter = 0
                package_coordinates = []
                volume_space = np.copy(empty_space)
                continue
        
        # Increase counter to move to next package
        package_counter += 1
        # Once all packages have been placed
        if package_counter == len(package_list):
            # Score stacking
            score = score_space(volume_space, article_list)
            # Return
            if queue is not None:
                queue.put((score, attempts_counter, volume_space, package_coordinates))
            return score, attempts_counter, volume_space, package_coordinates

In [ ]:
optimizer(generate_package_lists(article_list, random_lists=0)[0], article_list, np.copy(volume_space))

### Set up threading

In [ ]:
def generate_optimizer(article_list, volume_space, generator_sorters = ["volume|ascending", "volume|ascending", "volume|descending"], generator_random_lists = 10, optimizer_max_attempts = 10):
    '''
    Main function to run this module.
    First checks if packages can fit at all, then generates various package lists.
    Runs optimizers in parallel on each list.
    Finds lowest achieved score.
    Returns filled volume_space and package coordinates.
    '''
    # Check if package volume is smaller than or equal to available space, otherwise return error
    if not is_space_sufficient(article_list, volume_space):
        return INSUFFICIENT_SPACE
    # Check if longest package dimension is smaller than or equal to longest space dimension, otherwise return error
    if not is_longest_dimension_sufficient(article_list, volume_space):
        return INSUFFICIENT_DIMENSION
    # Generate list of package lists
    package_lists = generate_package_lists(article_list, sorters=generator_sorters, random_lists=generator_random_lists)
    
    # Set up threads
    threads = []
    queue = Queue()
    return_vals = []
    # Call optimizer function for each package list
    for package_list in package_lists:
        # Set up new thread
        optimizer_thread = Thread(target=optimizer, args=(package_list, article_list, np.copy(volume_space), queue, optimizer_max_attempts))
        # Append to thread list
        threads.append(optimizer_thread)
        # Start thread
        optimizer_thread.start()
        response = queue.get()
        return_vals.append(response)
    # Receive return values back for each thread
    for idx, thread in enumerate(threads):
        thread.join()
    # Find lowest score
    scores = [return_val[0] for return_val in return_vals]
    score_index = scores.index(min(scores))
    # Return 
    return return_vals[score_index][2:]

In [ ]:
opt = generate_optimizer(article_list, np.copy(volume_space), generator_random_lists=1)